In [2]:
%matplotlib inline
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation, patches, rc
import ipywidgets as widgets
import matplotlib as mpl
rc('animation', html='html5')
rc('savefig', dpi=80)
rc('figure', dpi=80)
from IPython.display import YouTubeVideo, HTML, Audio

# Diseño de filtros IIR

Un filtro FIR de buena calidad puede requerir una gran cantidad de coeficientes

Es posible implementar filtros más eficientes usando recursividad

## Sistema IIR (infinite impulse response)

Generalizando el sistema FIR para incluir versiones pasadas de la salida y asumiendo $a[0] = 1$ llegamos a 

$$
\begin{align}
y[n] &= b[0] x[n] + b[1] x[n-1] + b[2] x[n-2] + \ldots + b[L] x[n-L]  \nonumber \\
& - a[1] y[n-1] - a[2] y[n-2] - \ldots - a[M] y[n-M] \nonumber \\
&= \sum_{l=0}^{L} b[l] x[n-l] - \sum_{m=1}^{M} a[m] y[n-m]  \nonumber  \\
\sum_{m=0}^{M} a[m] y[n-m] &= \sum_{l=0}^{L} b[l] x[n-l] \nonumber \\
(a * y)[n] &= (b * x)[n], \nonumber
\end{align}
$$

es decir dos convoluciones discretas que definen una **ecuación de diferencias**

Este tipo de sistema se conoce como 
- sistema IIR (infinite impulse response)
- sistema *auto-regresive moving average* (ARMA)
    - autoregresivo de orden M: incluye valores pasados de la salida
    - media movil de orden L+1: pondera el valor presente y pasados de la entrada

¿Cuando se recupera un sistema FIR?

R: Si $a[m] = 0$ para $m=[1, \ldots, M]$

## Respuesta en frecuencia del sistema IIR

Aplicando la transformada de Fourier convertimos las convoluciones en multiplicaciones y encontramos la respuesta en frecuencia como

$$
\begin{align}
\text{DFT}_N[(a * y)[n]] &= \text{DFT}_N[(b * x)[n]] \nonumber \\
A[k] Y[k] &= B[k] X[k] \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{B[k]}{A[k]} = \frac{ \sum_{l=0}^L b[l]e^{-j \frac{2\pi}{N} nl} }{ \sum_{m=0}^M a[m]e^{-j \frac{2\pi}{N} mk}} \nonumber
\end{align}
$$
siempre que $A[k] \neq 0$. 

Se llega a un resultado equivalente usando la propiedad de traslación de la DFT

$$
\begin{align}
\text{DFT}_N\left[\sum_{m=0}^{M} a[m] y[n-m]\right] &= \text{DFT}_N\left[\sum_{l=0}^{L} b[l] x[n-l]\right] \nonumber \\
\sum_{m=0}^{M} a[m] \mathbb{DFT}_N\left[y[n-m]\right] &= \sum_{l=0}^{L} b[l] \mathbb{DFT}_N\left[x[n-l]\right] \nonumber \\
\sum_{m=0}^{M} a[m] Y[k] e^{-j \frac{2\pi}{N} km} &= \sum_{l=0}^{L} b[l] X[k] e^{-j \frac{2\pi}{N} kl} \nonumber \\
Y[k] \sum_{m=0}^{M} a[m] e^{-j \frac{2\pi}{N} km} &= X[k] \sum_{l=0}^{L} b[l] e^{-j \frac{2\pi}{N} kl} \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{ \sum_{l=0}^L b[l]e^{-j \frac{2\pi}{N} lk} }{ \sum_{m=0}^M a[m]e^{-j \frac{2\pi}{N} mk}} \nonumber
\end{align}
$$

que también puede expresarse de forma alternativa como 

$$
H[k] = K \frac{ \prod_{l=1}^L (e^{j \frac{2\pi}{N} k}- \beta[l]) }{ \prod_{m=1}^M (e^{j \frac{2\pi}{N} k}- \alpha[m])} 
$$

donde $K$ es la **ganancia** y las raices de los polinomios se llaman **ceros** (numerador) y **polos** (denominador)

### Ejemplo:

Consideremos el siguiente sistema IIR 

$$
\begin{align}
y[n] &= (1-\gamma) x[n] + \gamma y[n-1] \nonumber \\
y[n] - \gamma y[n-1] &= (1-\gamma) x[n] \nonumber
\end{align}
$$

¿Cuántos coeficientes tiene este sistema?

R: $a[0] = 1$, $a[1] = -\gamma$ y $b[0] = (1-\gamma)$, es decir MA de orden 1 y AR de orden 1

¿Cúal es su respuesta al impulso? (asumiendo $y[n]=0, n<0$)

$$
\begin{matrix}
n & \delta[n] & y[n] \\
-2 & 0 & 0 \\
-1 & 0 & 0 \\
0 & 1 & (1-\gamma) \\
1 & 0 & \gamma(1-\gamma) \\
2 & 0 & \gamma^2(1-\gamma)  \\
3 & 0 & \gamma^3(1-\gamma) \\
4 & 0 & \gamma^4(1-\gamma) \\
\end{matrix}
$$

¿Qué pasa si $\gamma \geq 1$?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, figsize=(9, 5))

def update(gamma):
    t, y = scipy.signal.dimpulse(([1-gamma, 0], [1, -gamma], 1), x0=0, n=30)
    ax.cla(); ax.plot(t, y[0], lw=4); 

interact(update, gamma=FloatSlider_nice(min=-1.1, max=2.1, step=0.01), msg_throttle=1);

- La respuesta al impulso puede tender a infinito!
- A diferencia de un sistema FIR el sistema IIR puede tener una configuración
    - Estable: $h$ tiende a cero
    - Condicionalmente estable: $h$ oscila a una taza constante
    - Inestable: $h$ crece/decrece infinitamente

Consideremos el sistema anterior y asumamos que $|\gamma|<1$, desenrollando tenemos que 

$$
\begin{align}
y[0] &= (1-\gamma) x[0] \nonumber \\
y[1] &= (1-\gamma) (x[1] + \gamma x[0]) \nonumber \\
y[2] &= (1-\gamma) (x[2] + \gamma x[1] + \gamma^2 x[0]) \nonumber \\
y[3] &= (1-\gamma) (x[3] + \gamma x[2] + \gamma^2 x[1] + \gamma^3 x[0]) \nonumber \\
y[4] &= (1-\gamma) (x[4] + \gamma x[3] + \gamma^2 x[2] + \gamma^3 x[1]  + \gamma^4 x[0]) \nonumber \\
y[5] &= \ldots \nonumber 
\end{align}
$$

Con un sistema IIR de 3 coeficientes podemos representar un sistema FIR considerablemente más grande

Por ejemplo si escogemos $\gamma$ tal que $\gamma^{20 }\approx 0$ entonces aproximamos un sistema FIR de orden 20

Por otra parte la respuesta en frecuencia de este sistema

$$
\begin{align}
Y[k] &= (1-\gamma) X[k] + \gamma Y[k] e^{-j \frac{2\pi}{N} k} \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{1-\gamma}{1 - \gamma e^{-j \frac{2\pi}{N} k} } = (1-\gamma)\frac{e^{j \frac{2\pi}{N} k} - 0}{e^{j \frac{2\pi}{N} k} - \gamma  } \nonumber 
\end{align}
$$

que tiene un cero en $0$, un polo en $\gamma$ y ganancia $1-\gamma$

Para entender mejor este sistema estudiemos la magnitud de $|H[k]|$ para $\gamma < 1$

$$
\begin{align}
| H[k]| &= \frac{|1-\gamma|}{|1 - \gamma e^{-j \frac{2\pi}{N} k}|}  \nonumber \\
&=  \frac{1-\gamma}{\sqrt{1 - 2\gamma \cos(\frac{2\pi}{N} k) + \gamma^2}} \nonumber
\end{align}
$$

¿Cómo se ve $|H[k]|$? ¿Qué función cumple este sistema?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, figsize=(9, 5))
k = np.arange(-24, 25)/50
Hk = lambda gamma, k : (1-gamma)/np.sqrt(1 - 2*gamma*np.cos(2.0*np.pi*k) + gamma**2)
l = ax.plot(k, np.zeros_like(k), lw=4); ax.set_ylim([-0.05, 1.05]); ax.set_xlabel('Frequency')
def update(gamma):
    l[0].set_ydata(Hk(gamma, k));
    ax.set_title(r"$\gamma$: %0.2f" %(gamma))
interact(update, gamma=FloatSlider_nice(min=0., max=0.99, step=0.01), msg_throttle=1);

- Este sistema actua como filtro pasa bajo!

## Diseño de filtros IIR simples

Los filtros IIR más simples son los de un polo y un cero

$$
H[k] = \frac{b[0] + b[1] e^{-j \frac{2\pi}{N} k}}{1 + a[1] e^{-j \frac{2\pi}{N} k}}  = K\frac{e^{j \frac{2\pi}{N} k} - \beta}{e^{j \frac{2\pi}{N} k} - \alpha  } 
$$

donde podemos reconocer $b[0]=K$, $\beta = - b[1]K$ y $\alpha=-a[1]$

- Definimos la frecuencia de corte $f_c$ como aquella frecuencia en la que el filtro alcanza una atenuación de 0.7 (-3 dB)
- En este caso $\gamma = e^{-2\pi f_c}$

**Receta para un filtro pasa bajo IIR con frecuencia de corte $f_c$**

- $b[0] = 1 - e^{-2\pi f_c}$
- $b[1] = 0$
- $a[1] = -e^{-2\pi f_c}$

$$
H[k] = \frac{1-e^{-2\pi f_c}}{1 - e^{-2\pi f_c} e^{-j \frac{2\pi}{N} k}} = (1-e^{-2\pi f_c}) \frac{(e^{j \frac{2\pi}{N} k}- 0)}{(e^{j \frac{2\pi}{N} k} - e^{-2\pi f_c} )}
$$
- Un cero en $0$, un polo en $e^{-2\pi f_c}$ y ganancia $1-e^{-2\pi f_c}$

**Receta para un filtro pasa alto IIR con frecuencia de corte $f_c$**
- $b[0] = (1 + e^{-2\pi f_c})/2$
- $b[1] = -(1 + e^{-2\pi f_c})/2$
- $a[1] = -e^{-2\pi f_c}$

$$
H[k] = \frac{1+e^{-2\pi f_c}}{2} \frac{(e^{j \frac{2\pi}{N} k} - 1)}{(e^{j \frac{2\pi}{N} k} - e^{-2\pi f_c})}
$$
- Un cero en $1$, un polo en $e^{-2\pi f_c}$ y ganancia $\frac{1+e^{-2\pi f_c}}{2}$



In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(9, 7))

def update(fc):
    gamma = np.exp(2*np.pi*fc)
    b, a = [(1-gamma), 0], [1, -gamma]
    #gamma = np.exp(-2*np.pi*(fc))
    #b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]
    freq, response = scipy.signal.freqz(b, a)
    ax[0].cla(); ax[0].plot(0.5*freq/np.pi, np.abs(response))
    ax[0].set_ylabel('Espectro de\n magnitud')
    ax[1].cla(); ax[1].plot(0.5*freq/np.pi, np.unwrap(np.angle(response)));
    ax[1].set_ylabel('Espectro de\n ángulo')
    freq, delay = scipy.signal.group_delay(system=(b, a))
    ax[2].cla(); ax[2].plot(0.5*freq/np.pi, delay); ax[2].set_ylabel("Retardo")

interact(update, fc=FloatSlider_nice(min=0.01, max=0.49, step=0.01, value=0.01));

- Su retardo no es igual para todas las frecuencias! ¿Qué consecuencia tiene esto?


Aplicar un filtro a una señal:

`scipy.signal.lfilter(b, a, x, axis=-1, zi=None)`
- b (arreglo): coeficientes del numerador
- a (arreglo): coeficientes del denominador
- x (arreglo): señal de entrada

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 7))
fc = 0.05; gamma = np.exp(-2*np.pi*(fc))
# b, a = [(1-gamma), 0], [1, -gamma]  # Low Pass
b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]  # High pass
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); ax[0].set_ylim([-0.05, 1.05]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n)); ax[1].set_ylim([-0.5, 1.05]); ax[1].set_title('Output')

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)
    l0[0].set_ydata(x);
    y = scipy.signal.lfilter(b, a, x)
    l1[0].set_ydata(y);

anim = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True); 
#plt.close(); HTML(anim.to_html5_video())

## Diseño de filtros IIR de segundo orden

Los filtros IIR de segundo orden o **biquad** tienen dos polos y dos ceros.

Su respuesta en frecuencia es
$$
H[k] = \frac{b[0] + b[1] W_N^k + b[2] W_N^{2k}}{1 + a[1] W_N^k + a[2] W_N^{2k}} = K \frac{(W_N^{-k} - \beta_1) (W_N^{-k} - \beta_2)}{(W_N^{-k} - \alpha_1)(W_N^{-k} - \alpha_2)},
$$
donde $W_N = e^{-j \frac{2 \pi}{N}}$ y la relación entreo coeficientes y polos/ceros es: 
$$
b[0] = K, \quad b[1] = -K (\beta_1 + \beta_2), \quad b[2]= K \beta_1\beta_2
$$
$$
a[1] = - (\alpha_1 + \alpha_2), \quad a[2]=\alpha_1 \alpha_2
$$


- Con arquitecturas de segundo orden se pueden crear filtros pasabanda y rechaza banda
- Los filtros IIR de primer y segundo orden son menos propensos a inestabilidades
- https://www.dsprelated.com/showarticle/1137.php


In [ ]:
#https://www.slideshare.net/op205/basics-of-digital-filters-presentation

plt.close('all'); fig, ax = plt.subplots(3, figsize=(8, 6))

def update(fc1, fc2):
    gamma1 = np.exp(-2*np.pi*fc1)
    # gamma2 = np.exp(-2*np.pi*fc2)
    # b, a = [(1-gamma), 0], [1, -gamma]
    #b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]
    z = [1, -1]
    p = [0.9*np.exp(-1j*2*np.pi*fc1), 0.9*np.exp(1j*2*np.pi*fc1)]
    k=1
    freq, response = scipy.signal.freqz_zpk(z=z, k=k, p=p)
    ax[0].cla(); ax[0].plot(0.5*freq/np.pi, np.abs(response))
    ax[0].set_ylabel('Espectro de\n magnitud')
    ax[1].cla(); ax[1].plot(0.5*freq/np.pi, np.unwrap(np.angle(response)));
    ax[1].set_ylabel('Espectro de\n ángulo')
    #freq, delay = scipy.signal.group_delay(system=(b, a))
    #ax[2].cla(); ax[2].plot(0.5*freq/np.pi, delay); ax[2].set_ylabel("Retardo")

interact(update, fc1=FloatSlider_nice(min=0.01, max=0.49, step=0.01, value=0.25),
         fc2=FloatSlider_nice(min=0.01, max=0.49, step=0.01, value=0.3));

## Diseño de filtros IIR de orden mayor

Crear coeficientes de filtro IIR de orden mayor

`scipy.signal.iirfilter(N, Wn, rp=None, rs=None, btype='bandpass', analog=False, ftype='butter', output='ba')`

- N (entero): orden del filtro
- Wn (arreglo): Frecuencias de corte (normalizadas en [0, 1])
- btype (string): Tipo de filtro {'bandpass', 'lowpass', 'highpass', 'bandstop'}
- ftype (string): Tipo de filtro {'butter', 'ellip', 'cheby1', 'cheby2', 'bessel'}
- output (string): Tipo de retorno

El filtro Butterworth ('butter') es óptimo en el sentido de tener la banda de paso lo más plana posible. Otros filtros se diseñaron con otras consideraciones. Los filtros IIR digitales están basados en los filtros IIR analógicos.

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(9, 7))

def update(order, ftype):
    b, a = scipy.signal.iirfilter(N=order, Wn=0.2, ftype=ftype, btype='lowpass', output='ba')
    freq, response = scipy.signal.freqz(b, a)
    ax[0].cla(); ax[0].plot(freq/np.pi, np.abs(response))
    ax[0].set_ylabel('Espectro\nde magnitud')
    ax[1].cla(); ax[1].plot(freq/np.pi, np.unwrap(np.angle(response)));
    ax[1].set_ylabel('Espectro\nde ángulo')
    freq, delay = scipy.signal.group_delay(system=(b, a))
    ax[2].cla(); ax[2].plot(freq/np.pi, delay); ax[2].set_ylabel("Retardo")
    
interact(update, order=SelectionSlider_nice(options=[1, 2, 3, 5, 10, 15, 25], value=1),
         ftype=SelectionSlider_nice(options=['butter', 'bessel']));

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 6))

b, a = scipy.signal.iirfilter(N=10, Wn=0.03, ftype='butter', btype='lowpass', output='ba')
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); ax[0].set_ylim([-0.05, 1.05]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n)); ax[1].set_ylim([-0.2, 1.2]); ax[1].set_title('Output')

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)
    l0[0].set_ydata(x);
    y = scipy.signal.lfilter(b, a, x)
    #y = scipy.signal.filtfilt(b, a, x)
    l1[0].set_ydata(y);

anim = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True); 
#plt.close(); HTML(anim.to_html5_video())

## Respuesta en frecuencia de filtros FIR e IIR del orden equivalente

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, 2, figsize=(9, 6))
b, a = scipy.signal.iirfilter(N=20, Wn=0.2, ftype='butter', btype='lowpass', output='ba')
h = scipy.signal.firwin(numtaps=20, cutoff=0.2, pass_zero=True, window='hann', fs=2)
freq_fir, response_fir = scipy.signal.freqz(h, 1)
freq_iir, response_iir = scipy.signal.freqz(b, a)

ax[0, 0].set_title('Espectro\nde magnitud'); ax[0, 1].set_title('Retardo')
ax[0, 0].plot(freq_fir/np.pi, np.abs(response_fir))
ax[1, 0].plot(freq_iir/np.pi, np.abs(response_iir)); 
ax[0, 0].set_ylabel('FIR'); ax[1, 0].set_ylabel('IIR'); 
freq_fir, delay_fir = scipy.signal.group_delay(system=(h, 1))
ax[0, 1].plot(freq_fir/np.pi, delay_fir); ax[0, 1].set_ylim([np.mean(delay_fir)-1, np.mean(delay_fir)+1])
freq_iir, delay_iir = scipy.signal.group_delay(system=(b, a))
ax[1, 1].plot(freq_iir/np.pi, delay_iir); 

## Efectos de audio: <a href="https://en.wikipedia.org/wiki/Wah-wah_(music)">Wah-wah</a> 

- Filtro pasabanda modulado
- Ancho de pasada fijo $f_b$ [Hz]
- Frecuencia central variable $f_c$ [Hz]
- La frecuencia central se modula con una onda lenta
- Se modela como el siguiente sistema **IIR**
$$
H[k] = \frac{(1+c)W_N^{2k} -(1+c) }{W_N^{2k} + d(1-c)W_N^k -c}
$$
donde $d=-\cos(2\pi f_c/f_s)$ y $c = \frac{\tan(\pi f_b/f_s) -1}{\tan(2\pi f_b /f_s)+1}$



Ref: https://www.ee.columbia.edu/~ronw/adst-spring2010/lectures/lecture2.pdf

In [ ]:
data, fs = sf.read("data/nomekop.ogg")
Audio(data, rate=int(fs))

In [ ]:
fb, N, Nw = 200, len(data), 5
c  = (np.tan(np.pi*fb/fs) - 1.)/(np.tan(2*np.pi*fb/fs) +1)
data_wah = []
zi = np.zeros(shape=(2,))
for k in range(int(N/Nw)+1):
    fc = 500 + 2000*(np.cos(2.0*np.pi*k*30./fs) +1)/2
    d = -np.cos(2*np.pi*fc/fs)
    data2, zi = scipy.signal.lfilter([(1+c), 0, -(1+c)], [1, d*(1-c), -c], 
                                     x=data[k*Nw:(k+1)*Nw], zi=zi)
    data_wah.append(data2)
Audio(np.hstack(data_wah), rate=int(fs))

Design of recursive filters
http://www-pagines.fib.upc.es/~pds/Lect06.pdf